# Data Engeneering Project 2

### Requiered Libraries

In [1]:
!pip install unidecode
!pip install gensim
!pip install nltk==3.4

### Importing libraries

In [2]:
import pandas as pd
import nltk
import re
#from nltk.tokenize import RegexpTokenize
from nltk.tokenize import TweetTokenizer
from nltk import word_tokenize
from nltk.corpus import stopwords
from unidecode import unidecode
import string
import gensim
from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer


In [3]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Massarata\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Massarata\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Massarata\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Load data

In [32]:
df=pd.read_csv('tweets.csv')

### Preprocessing

#### Removing noise 

In [19]:
def remove_noise(a):
    
    if "pic" in a:
        a=a[:-26]
    a=re.sub(r'(\s)#\w+', r'\1', a)
    tokenizer = RegexpTokenizer(r'\w+')
    a=tokenizer.tokenize(a)
    a=" ".join(str(x) for x in a)
    a=re.sub('[0-9]+', '', a)
    
    return a
    
    

#### Stop words and tokenization

In [20]:
def pre_process(corpus):
    # convert input corpus to lower case.
    corpus = corpus.lower()
    # collecting a list of stop words from nltk and punctuation form
    # string class and create single array.
    stopset = stopwords.words('english') + list(string.punctuation)
    # remove stop words and punctuations from string.
    # word_tokenize is used to tokenize the input corpus in word tokens.
    corpus = " ".join([i for i in word_tokenize(corpus) if i not in stopset])
    # remove non-ascii characters
    corpus = unidecode(corpus)
    return corpus

#### Lemmatization

In [21]:
from nltk.stem import WordNetLemmatizer
def lemmatize (sentence):
    lemmatizer = WordNetLemmatizer()
    words = word_tokenize(sentence)
  
    out = " ".join([lemmatizer.lemmatize(w) for w in words ])
  
    return(out)

In [33]:
df['text_proced']=df['text'].apply(remove_noise)
df['text_proced']=df['text_proced'].apply(pre_process)
df['text_proced']=df['text_proced'].apply(lemmatize)



In [34]:
df

,Unnamed: 0,date,id,link,retweet,text,author,text_proced
0,0,Oct 7,784609194234306560,/realDonaldTrump/status/784609194234306560,False,Here is my statement.pic.twitter.com/WAZiGoQqMQ,DonaldTrump,statement
1,1,Oct 10,785608815962099712,/realDonaldTrump/status/785608815962099712,False,Is this really America? Terrible!pic.twitter.c...,DonaldTrump,really america terrible
2,2,Oct 8,784840992734064640,/realDonaldTrump/status/784840992734064641,False,The media and establishment want me out of the...,DonaldTrump,medium establishment want race badly never dro...
3,3,Oct 8,784767399442653184,/realDonaldTrump/status/784767399442653184,False,Certainly has been an interesting 24 hours!,DonaldTrump,certainly interesting hour
4,4,Oct 10,785561269571026944,/realDonaldTrump/status/785561269571026946,False,Debate polls look great - thank you!\n#MAGA #A...,DonaldTrump,debate poll look great thank
...,...,...,...,...,...,...,...,...
17211,17211,12 May 2009,1773561338,/realDonaldTrump/status/1773561338,False,"""My persona will never be that of a wallflower...",DonaldTrump,persona never wallflower rather build wall cli...
17212,17212,8 May 2009,1741160716,/realDonaldTrump/status/1741160716,False,New Blog Post: Celebrity Apprentice Finale and...,DonaldTrump,new blog post celebrity apprentice finale less...
17213,17213,8 May 2009,1737479987,/realDonaldTrump/status/1737479987,False,Donald Trump reads Top Ten Financial Tips on L...,DonaldTrump,donald trump read top ten financial tip late s...
17214,17214,4 May 2009,1701461182,/realDonaldTrump/status/1701461182,False,Donald Trump will be appearing on The View tom...,DonaldTrump,donald trump appearing view tomorrow morning d...


### Transfer learning : updation the model from GoogleNews

In [35]:
tokenizer =RegexpTokenizer(r'\w+')
sentences=[tokenizer.tokenize(df['text_proced'][i]) for i in range(len(df['text_proced']))]


# size option needs to be set to 300 to be the same as Google's pre-trained model
 
model = Word2Vec(size = 300, window=5,
min_count = 1, workers = 2)
 
model.build_vocab(sentences)
 
# assign the vectors to the vocabs that are in Google's pre-trained model and your sentences defined above.
 
# lockf needs to be set to 1.0 to allow continued training.
 
model.intersect_word2vec_format('GoogleNews-vectors-negative300.bin', lockf=1.0, binary=True)
 
# continue training with you own data
 
model.train(sentences, total_examples=3, epochs = 5)


(878148, 946125)

In [36]:
sentences

[['statement'],
 ['really', 'america', 'terrible'],
 ['medium',
  'establishment',
  'want',
  'race',
  'badly',
  'never',
  'drop',
  'race',
  'never',
  'let',
  'supporter'],
 ['certainly', 'interesting', 'hour'],
 ['debate', 'poll', 'look', 'great', 'thank'],
 ['saying',
  'clinton',
  'campaign',
  'anti',
  'catholic',
  'bigotry',
  'http',
  'bit',
  'ly',
  'dcbtvkcrooked'],
 ['thank'],
 ['cincinnati',
  'ohio',
  'tomorrow',
  'night',
  'pm',
  'join',
  'ticket',
  'http',
  'www',
  'donaldjtrump',
  'com',
  'schedule',
  'register',
  'cincinnati',
  'oh'],
 ['little',
  'pick',
  'dishonest',
  'medium',
  'incredible',
  'information',
  'provided',
  'wikileaks'],
 ['thank',
  'florida',
  'movement',
  'never',
  'seen',
  'never',
  'seen',
  'let',
  'get'],
 ['foul',
  'mouthed',
  'sen',
  'john',
  'mccain',
  'begged',
  'support',
  'primary',
  'gave',
  'dropped',
  'locker',
  'room',
  'remark'],
 ['disloyal',
  'r',
  'far',
  'difficult',
  'crooked',

### Top 20 tweets

In [37]:
text=" time for a Royal Celebration!"
text=lemmatize(pre_process(remove_noise(text)))
i=0
data= pd.DataFrame([],columns=['tweet_index',"similarity_score"])
while i <len(df['text_proced']):
    a={"tweet_index":i,"similarity_score":model.wv.n_similarity(text.lower().split(), df['text_proced'][i].lower().split())}
    data=data.append(a,ignore_index=True)
    i=i+1
    top_20=data.nlargest(20, ['similarity_score'])
    



In [38]:
top_20

,tweet_index,similarity_score
17063,17063.0,0.706839
8631,8631.0,0.693853
8639,8639.0,0.691073
7317,7317.0,0.689680
4312,4312.0,0.677906
10588,10588.0,0.674086
12806,12806.0,0.672592
14866,14866.0,0.672592
8425,8425.0,0.667483
10163,10163.0,0.666663


In [39]:
print(" In order, this tweet is similar ")
for i in top_20['tweet_index']:
    print("at "+ "{:.0%}".format(top_20["similarity_score"][i])+" to the tweet of " + df.loc[i,"author"]+ " posted the "+df.loc[i,"date"]+" And the text is "+df.loc[i,"text"])

 In order, this tweet is similar 
at 71% to the tweet of DonaldTrump posted the 1 Jul 2011 And the text is Wishing everyone a wonderful Independence Day holiday weekend, a great celebration for a great country.
at 69% to the tweet of DonaldTrump posted the 28 Nov 2013 And the text is HAPPY THANKSGIVING EVERYONE, ENJOY YOUR DAY!
at 69% to the tweet of DonaldTrump posted the 27 Nov 2013 And the text is Make sure to enjoy your time with your family during the holiday. It is a special time. Love and appreciate your family.
at 69% to the tweet of DonaldTrump posted the 13 Jun 2014 And the text is "@nikkio: I know I'm a day early for your weekend celebrations-Happy Birthday Donald! Looking forward to your surprises for 2014-2015 Thanks
at 68% to the tweet of DonaldTrump posted the 12 Apr 2015 And the text is "@TalkerNewYorker: I say Flag Day perfect for an announcement of the presidential variety @realDonaldTrump"  My birthday, interesting.
at 67% to the tweet of DonaldTrump posted the 24 Ma